### Defining and importing constants

Inpired by rubys/feedvalidator at [line 55](https://lgtm.com/projects/g/rubys/feedvalidator/snapshot/eb77aa864016e1071f37357746a3ad77de94432a/files/src/rdflib/syntax/parsers/RDFXMLHandler.py?sort=name&dir=ASC&mode=heatmap#L55)

You can import another file as a module and use the variables within as constants.

In [4]:
from pyExamples.constants import C, PI, K, G

In [5]:
G, C # Gravitational constant, Speed of Light

(6.6743015e-11, 299792358)

But is it _really_ a constant?

In [6]:
G = 0

In [7]:
G

0

In spite of being from another module, you can't make it immutable.

`[?]` How do you make an imported variable immutable?  
`[ ]` Maybe create a constants class?

Lets define Rydberg constant as `RINF`:

In [8]:
RINF = 10e6 # painfully rough

Lets import the constant which holds a more accurate number.

In [9]:
from pyExamples.constants import RINF

RINF already exists but is overwritten. In the past I have seen error being thrown because of this. It doesn't seem to happen all the time but its worth remembering the potential clash.

In [10]:
RINF

10973731.56816021

Generally, you don't want to use `import`, especially `import *` as you are flooding your namespace with the `module`'s namespace. Thereby inadvertently overwriting and clashing stuff. So you want to do this:

In [11]:
import pyExamples.constants as constants
constants.RINF

10973731.56816021

Another subtlety is that if you change `RINF` you don't change `constants.RINF`. So you can reimport the `constants` module and recover the number (without restarting the nb kernel). However if you change `constants.RINF` you change the module's variable value instead, no importing will work [[so-import-variables](https://stackoverflow.com/questions/17255737/importing-variables-from-another-file)].

In [12]:
constants.RINF = 0
constants.RINF

0

In [13]:
RINF

10973731.56816021

`[?]` Is there a way to force the rereading of the file?  
`>>>` Maybe but you don't want to allow the change in the first place. You need to modify how the `constant` module's attributes are modified.

### Class Attributes based - Constants

You can see that a constant can be overwritten even when coming from another module like `constant.RINF = 0`. This is because the `property()` function is still defined and leaves the object subject being changed. The `property` function manages the getting, setting, deleting of properties in the `__dict__` of the object. You can read more about it in [[py-docs-prop](https://docs.python.org/3/library/functions.html#property)].

In [14]:
from pyExamples.constants import RINF
RINF

0

In `.src/const.py` there is a `const` class that holds all the attributes and manages then with its own get, set and delete attributes methods.

In [15]:
import pyExamples.const as const

Here the `__setattr__` built-in function is involved thanks to the assignment operator used against the `const.RINF`. The functions overwrites the default `__setattr__` behaviour with its own by checking if attribute name exists in the `const` class symbol table (`const.__dict__`). If it does it throwns a custom `ConstError` otherwise goes ahead and assigns the name, value pair in the `const` class `__dict__`.

[[py-docs-abc](https://docs.python.org/3/tutorial/errors.html#raising-exceptions)] Abstract base classes

In [16]:
const.RINF = RINF

In [17]:
const.RINF

0

If you reassign, the `__setattr__` is invoked again but this time, `RINF` already exists in the `const` class symbol table. Thereby throwing an error.

In [18]:
# const.RINF = 80
# _ConstError

`line 23-24` in `src.const.py`overwrite the `syst.module.const` with the hidden `_const` class. This means any attributes added to `const` are instances of the `_const` instead of the `const.py` module. It means you can then enforce the `__setattr` functions against those attributes. Since the class and the constant error `_ConstError` class is hidden, only the constants are available to see.

Read more about `sys.modules` [[py-docs-sysmod](https://docs.python.org/3/library/sys.html#sys.modules)]

In [19]:
type(const)

pyExamples.const._const

In [20]:
const.__dict__

{'RINF': 0}

`[ ]` Disable `line 23-24` in `const.py`, restart the kernel and rerun the cells above. What is the difference and why? How come `const.RINF` is mutable?

### Class function based - Constants

[[py-docs-typefinal](https://docs.python.org/3/library/typing.html#typing.Final)] `typing.Final`  
[[so-creating-const](https://stackoverflow.com/questions/2682745/how-do-i-create-a-constant-in-python?noredirect=1&lq=1) `Stackoverflow - Creating Constants`]

### Using `typing.final`

This is relatively new feature which was introduced in Python 3.8 [[pep-591](https://www.python.org/dev/peps/pep-0591/)].

In [21]:
from typing import final

In [22]:
JO: final = RINF

In [23]:
JO

0

Tthe `typing.Final` does not inhibit mutability but instead get warned using static type checkers like [[`mypy`](http://mypy-lang.org/examples.html)].

In [24]:
JO = 1

In [25]:
JO

1

`[?]` Is defining constants a step away from idiomatic python coding style?

### Using @property decorator

[[ffc-propdeco](https://www.freecodecamp.org/news/python-property-decorator/)] The @property Decorator in Python: It Use Cases, Advantages, and Syntax  
[[zhou-decolev](https://medium.com/techtofreedom/7-levels-of-using-decorators-in-python-370473fcbe76)] 7 Levels of Using Decorators  
[[so-creating-const](https://stackoverflow.com/questions/2682745/how-do-i-create-a-constant-in-python?noredirect=1&lq=1)] Stackoverflow - Creating Constants


In [26]:
help(property)

Help on class property in module builtins:

class property(object)
 |  property(fget=None, fset=None, fdel=None, doc=None)
 |  
 |  Property attribute.
 |  
 |    fget
 |      function to be used for getting an attribute value
 |    fset
 |      function to be used for setting an attribute value
 |    fdel
 |      function to be used for del'ing an attribute
 |    doc
 |      docstring
 |  
 |  Typical use is to define a managed attribute x:
 |  
 |  class C(object):
 |      def getx(self): return self._x
 |      def setx(self, value): self._x = value
 |      def delx(self): del self._x
 |      x = property(getx, setx, delx, "I'm the 'x' property.")
 |  
 |  Decorators make defining new properties or modifying existing ones easy:
 |  
 |  class C(object):
 |      @property
 |      def x(self):
 |          "I am the 'x' property."
 |          return self._x
 |      @x.setter
 |      def x(self, value):
 |          self._x = value
 |      @x.deleter
 |      def x(self):
 |          del s

`[?]` Can the `@property` decorator be used outside a class definition?

In [27]:
import pyExamples.propconst as constp

In [28]:
constp.PI

3.14

In [29]:
# constp.PI = 60 # Attribute Error

The `constp` allows you to define all your constants as functions in one place. It uses the more pythonic `@` decorator which gives a very concise expression. Unlike `const` you can't define a new constant on the fly as you are not altering the `fget` and `fset` functions of how the class attributes are managed. This means you can create a new attribute against the class which isn't a constant.

In [30]:
constp.ETA = 70

One could disallow that by defining a `__setattr__` as raising an error. Uncomment the `__setattr__` definition in the `propconst.py` file, reimport and rerun the statement above. But _feels_ this is less elegant.